In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Set the path to your dataset folders for training and testing
train_data_path = "E:\Thesis\dataset\Final Dataset\Training"
test_data_path = "E:\Thesis\dataset\Final Dataset\Testing"

In [3]:
# Define image dimensions and batch size
image_width, image_height = 512, 512
batch_size = 16

In [4]:
# Create an ImageDataGenerator for data augmentation and preprocessing
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [5]:
# Load and preprocess the training data
train_generator = data_generator.flow_from_directory(
    train_data_path,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 6000 images belonging to 2 classes.


In [6]:
# Load and preprocess the testing data
test_generator = data_generator.flow_from_directory(
    test_data_path,
    target_size=(image_width, image_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 1200 images belonging to 2 classes.


In [7]:
# Load the InceptionResNetV2 model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(image_width, image_height, 3))

# Add custom classification layers on top of the base model
model = tf.keras.models.Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(len(train_generator.class_indices), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# Calculate the training and testing steps per epoch
train_steps_per_epoch = train_generator.samples // batch_size
test_steps_per_epoch = test_generator.samples // batch_size

In [9]:
# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [10]:
# Train the model with early stopping
model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=100,
    validation_data=test_generator,
    validation_steps=test_steps_per_epoch,
    callbacks=[early_stopping]
)


Epoch 1/100
375/375 [==============================] - 189s 445ms/step - loss: 0.3152 - accuracy: 0.8698 - val_loss: 0.3140 - val_accuracy: 0.8983
Epoch 2/100
375/375 [==============================] - 125s 334ms/step - loss: 0.1915 - accuracy: 0.9243 - val_loss: 0.4372 - val_accuracy: 0.8700
Epoch 3/100
375/375 [==============================] - 125s 334ms/step - loss: 0.1660 - accuracy: 0.9363 - val_loss: 0.3819 - val_accuracy: 0.9083
Epoch 4/100
375/375 [==============================] - 125s 333ms/step - loss: 0.1283 - accuracy: 0.9490 - val_loss: 0.2451 - val_accuracy: 0.9025
Epoch 5/100
375/375 [==============================] - 125s 333ms/step - loss: 0.1223 - accuracy: 0.9547 - val_loss: 0.2206 - val_accuracy: 0.9125
Epoch 6/100
375/375 [==============================] - 125s 332ms/step - loss: 0.0984 - accuracy: 0.9615 - val_loss: 0.2682 - val_accuracy: 0.9308
Epoch 7/100
375/375 [==============================] - 125s 333ms/step - loss: 0.0994 - accuracy: 0.9620 - val_loss: 0

In [ ]:
# Evaluate the model on the test data
test_generator.reset()
predictions = model.predict(test_generator, steps=test_steps_per_epoch)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = test_generator.classes

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(test_generator, steps=test_steps_per_epoch)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

In [ ]:
# Calculate prediction accuracy
prediction_accuracy = accuracy_score(true_labels, predicted_labels)
print('Prediction Accuracy:', prediction_accuracy)

In [ ]:
# Create a dictionary to map label indices to label names
label_map = train_generator.class_indices


In [ ]:
print('Classification Report:')
print(classification_report(true_labels, predicted_labels, target_names=label_map))


In [ ]:
print('Confusion Matrix:')
confusion_mat=confusion_matrix(true_labels, predicted_labels)
print(confusion_mat)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
# Plot confusion matrix as a heatmap
labels = ["male", "female"]
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()
